In [ ]:
# Dataset load

from google.colab import files
uploaded = files.upload()

Saving GlobalLandTemperaturesByCountry.csv to GlobalLandTemperaturesByCountry.csv


# Raw dataset exploration

Original dataset has 577,462 rows and 4 columns

In [ ]:
df.shape

(577462, 4)

In [ ]:
import pandas as pd

df = pd.read_csv("/content/GlobalLandTemperaturesByCountry.csv")

df.tail()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
577457,2013-05-01,19.059,1.022,Zimbabwe
577458,2013-06-01,17.613,0.473,Zimbabwe
577459,2013-07-01,17.000,0.453,Zimbabwe
577460,2013-08-01,19.759,0.717,Zimbabwe
577461,2013-09-01,NaN,NaN,Zimbabwe


In [ ]:
df.columns.tolist()

['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'Country']

We need to understand how many countries are in our dataset. The goal is to map these countries to contients so we can do our continent-focused models

Based on analysis, we have 243 total countries

In [ ]:
unique_countries = sorted(df["Country"].unique())
len(unique_countries), unique_countries

(243,
 ['Afghanistan',
  'Africa',
  'Albania',
  'Algeria',
  'American Samoa',
  'Andorra',
  'Angola',
  'Anguilla',
  'Antarctica',
  'Antigua And Barbuda',
  'Argentina',
  'Armenia',
  'Aruba',
  'Asia',
  'Australia',
  'Austria',
  'Azerbaijan',
  'Bahamas',
  'Bahrain',
  'Baker Island',
  'Bangladesh',
  'Barbados',
  'Belarus',
  'Belgium',
  'Belize',
  'Benin',
  'Bhutan',
  'Bolivia',
  'Bonaire, Saint Eustatius And Saba',
  'Bosnia And Herzegovina',
  'Botswana',
  'Brazil',
  'British Virgin Islands',
  'Bulgaria',
  'Burkina Faso',
  'Burma',
  'Burundi',
  'Cambodia',
  'Cameroon',
  'Canada',
  'Cape Verde',
  'Cayman Islands',
  'Central African Republic',
  'Chad',
  'Chile',
  'China',
  'Christmas Island',
  'Colombia',
  'Comoros',
  'Congo',
  'Congo (Democratic Republic Of The)',
  'Costa Rica',
  'Croatia',
  'Cuba',
  'Curaçao',
  'Cyprus',
  'Czech Republic',
  "Côte D'Ivoire",
  'Denmark',
  'Denmark (Europe)',
  'Djibouti',
  'Dominica',
  'Dominican Repu

In [ ]:
len(unique_countries)

243

Through initial review, it seems like we do have majority of countries. However, there are some continents in the dataset and also regions (e.g. islands) that are not directly countries but related to them

The goal would be to make continent = continent in the code (e.g. Africa = Africa) and link each region to its corresponding continent.

For country specifically, we can easily use a package or dataset to map them with the corresponding continent



Here, we are focusing on continent and trying to understand how many values are specifically for the corresponding continents.

We found there are 15,167 values for the 7 specific continents in the country columns

In [ ]:
continents = {
    "Africa", "Asia", "Europe", "North America",
    "South America", "Oceania", "Antarctica"
}

unique_vals = set(df["Country"].unique())
present_continents = unique_vals & continents

print("Continents present:", present_continents)
print("Number of continent entries:", len(present_continents))



Continents present: {'North America', 'South America', 'Oceania', 'Antarctica', 'Africa', 'Europe', 'Asia'}
Number of continent entries: 7


In [ ]:
df_continents = df[df["Country"].isin(continents)]
len(df_continents)

15167

In [ ]:
df_continents["Country"].value_counts()

,count
Country,
Europe,3239
North America,2941
Asia,2370
Africa,1965
South America,1953
Oceania,1935
Antarctica,764


Here we wanted to understand how many values there are for each country (this is minus the continents)

In [ ]:
df_countries = df[~df["Country"].isin(continents)]
country_counts = df_countries["Country"].value_counts()
country_counts

,count
Country,
Åland,3239
Albania,3239
Andorra,3239
Austria,3239
Belarus,3239
...,...
Federated States Of Micronesia,1427
Guam,1329
Northern Mariana Islands,1329


These are all the unique country names (plus the continents and regions)

In [ ]:
unique_countries = sorted(df["Country"].unique())
len(unique_countries), unique_countries  # show first 20

(243,
 ['Afghanistan',
  'Africa',
  'Albania',
  'Algeria',
  'American Samoa',
  'Andorra',
  'Angola',
  'Anguilla',
  'Antarctica',
  'Antigua And Barbuda',
  'Argentina',
  'Armenia',
  'Aruba',
  'Asia',
  'Australia',
  'Austria',
  'Azerbaijan',
  'Bahamas',
  'Bahrain',
  'Baker Island',
  'Bangladesh',
  'Barbados',
  'Belarus',
  'Belgium',
  'Belize',
  'Benin',
  'Bhutan',
  'Bolivia',
  'Bonaire, Saint Eustatius And Saba',
  'Bosnia And Herzegovina',
  'Botswana',
  'Brazil',
  'British Virgin Islands',
  'Bulgaria',
  'Burkina Faso',
  'Burma',
  'Burundi',
  'Cambodia',
  'Cameroon',
  'Canada',
  'Cape Verde',
  'Cayman Islands',
  'Central African Republic',
  'Chad',
  'Chile',
  'China',
  'Christmas Island',
  'Colombia',
  'Comoros',
  'Congo',
  'Congo (Democratic Republic Of The)',
  'Costa Rica',
  'Croatia',
  'Cuba',
  'Curaçao',
  'Cyprus',
  'Czech Republic',
  "Côte D'Ivoire",
  'Denmark',
  'Denmark (Europe)',
  'Djibouti',
  'Dominica',
  'Dominican Repu

# Mapping Methodology

Thep plan is to map country, continent, and region to the specific continent column using pycountry-convert package

Sources:

1.   https://github.com/jefftune/pycountry-convert
2.   https://pypi.org/project/pycountry-convert/
3.   https://www.youtube.com/watch?v=VMIRK36d-EY



Name Cleaning

In [ ]:
import re

def clean_name(name):
    name = re.sub(r"\s*\(.*?\)", "", name)  # remove anything in parentheses
    return name.strip()

In [ ]:
df["CleanName"] = df["Country"].apply(clean_name)

Installing the pycountry-convert package

In [ ]:
!apt-get install -y libgeos-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgeos-dev is already the newest version (3.12.1-1~jammy0).
libgeos-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
!pip install pycountry
!pip install wheel
!pip install --no-cache-dir shapely==1.8.5.post1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [ ]:
!pip install pycountry_convert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.3/252.3 kB 8.3 MB/s eta 0:00:00


In [ ]:
import pycountry_convert as pc # here we are just testing to make sure the package is working
pc.country_name_to_country_alpha2("France")

'FR'

Here we are mapping the countries/regions where pycountry-convert may not recognize

In [ ]:
import re
import pycountry
import pycountry_convert as pc

# 1. Continent names (direct matches)
continent_names = {
    "Africa", "Asia", "Europe",
    "North America", "South America",
    "Oceania", "Antarctica"
}

# 2. Manual mapping for regions, territories, islands, special cases
manual_map = {
    "Hong Kong": "Asia",
    "Macau": "Asia",
    "Greenland": "North America",
    "French Guiana": "South America",
    "Guadeloupe": "North America",
    "Martinique": "North America",
    "Curaçao": "North America",
    "Bonaire, Saint Eustatius And Saba": "North America",
    "Sint Maarten": "North America",
    "Puerto Rico": "North America",
    "New Caledonia": "Oceania",
    "French Polynesia": "Oceania",
    "Northern Mariana Islands": "Oceania",
    "American Samoa": "Oceania",
    "Kingman Reef": "Oceania",
    "Palmyra Atoll": "Oceania",
    "Falkland Islands (Islas Malvinas)": "South America",
    "Heard Island And Mcdonald Islands": "Antarctica",
    "French Southern And Antarctic Lands": "Antarctica",
    "Mayotte": "Africa",
    "Reunion": "Africa",
    "Western Sahara": "Africa",
    "Gaza Strip": "Asia",
    "Palestina": "Asia",
}

# 3. Helper function — returns the continent
def get_continent(name):
    # Remove parentheses: "France (Europe)" → "France"
    clean = re.sub(r"\s*\(.*?\)", "", name).strip()

    # Case 1 — The value itself IS a continent
    if clean in continent_names:
        return clean

    # Case 2 — Special territories/regions
    if clean in manual_map:
        return manual_map[clean]

    # Case 3 — Use pycountry_convert for normal countries
    try:
        alpha2 = pc.country_name_to_country_alpha2(clean)
        continent_code = pc.country_alpha2_to_continent_code(alpha2)
        return pc.convert_continent_code_to_continent_name(continent_code)
    except:
        pass

    # Case 4 — Try alternative spellings via pycountry
    try:
        country = pycountry.countries.lookup(clean)
        alpha2 = country.alpha_2
        continent_code = pc.country_alpha2_to_continent_code(alpha2)
        return pc.convert_continent_code_to_continent_name(continent_code)
    except:
        pass

    # Case 5 — No match at all
    return "Unknown"

In [ ]:
df["Continent"] = df["Country"].apply(get_continent)

There are a few countries that have Unknown for continent. We need to manually account for these as well (17,711 NAs)

In [ ]:
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country,CleanName,Continent
0,1743-11-01,4.384,2.294,Åland,Åland,Unknown
1,1743-12-01,NaN,NaN,Åland,Åland,Unknown
2,1744-01-01,NaN,NaN,Åland,Åland,Unknown
3,1744-02-01,NaN,NaN,Åland,Åland,Unknown
4,1744-03-01,NaN,NaN,Åland,Åland,Unknown


In [ ]:
df["Continent"].value_counts()

,count
Continent,
Europe,169473
Africa,118213
Asia,114866
North America,91923
Oceania,31835
South America,31101
Unknown,17711
Antarctica,2340


In [ ]:
unknown_locations = df[df["Continent"] == "Unknown"]["Country"].unique()
unknown_locations

array(['Åland', 'Baker Island', 'Burma', 'Guinea Bissau',
       'South Georgia And The South Sandwich Isla', 'Timor Leste',
       'Turks And Caicas Islands', 'Virgin Islands'], dtype=object)

In [ ]:
manual_map.update({
    "Åland": "Europe",
    "Baker Island": "Oceania",
    "Burma": "Asia",
    "Guinea Bissau": "Africa",
    "South Georgia And The South Sandwich Isla": "Antarctica",
    "Timor Leste": "Asia",
    "Turks And Caicas Islands": "North America",
    "Virgin Islands": "North America"
})

In [ ]:
df["Continent"] = df["Country"].apply(get_continent)

In [ ]:
unknown_locations = df[df["Continent"] == "Unknown"]["Country"].unique() # this val
unknown_locations

array([], dtype=object)

This is the updated values with the manual mapping per continent

In [ ]:
df["Continent"].value_counts()

,count
Continent,
Europe,172712
Africa,120190
Asia,118864
North America,96489
Oceania,34100
South America,31101
Antarctica,4006


In [ ]:
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country,CleanName,Continent
0,1743-11-01,4.384,2.294,Åland,Åland,Europe
1,1743-12-01,NaN,NaN,Åland,Åland,Europe
2,1744-01-01,NaN,NaN,Åland,Åland,Europe
3,1744-02-01,NaN,NaN,Åland,Åland,Europe
4,1744-03-01,NaN,NaN,Åland,Åland,Europe


We need to find gaps in df dataset

In [ ]:
df["dt"] = pd.to_datetime(df["dt"])
df = df.sort_values(["CleanName", "dt"])

In [ ]:
dupes = df[df.duplicated(subset=["CleanName", "dt"], keep=False)]
dupes.head(20)

,dt,AverageTemperature,AverageTemperatureUncertainty,Country,CleanName,Continent
119802,1856-01-01,23.685,0.865,Congo (Democratic Republic Of The),Congo,Africa
121695,1856-01-01,24.407,0.880,Congo,Congo,Africa
119803,1856-02-01,23.162,1.241,Congo (Democratic Republic Of The),Congo,Africa
121696,1856-02-01,24.231,1.455,Congo,Congo,Africa
119804,1856-03-01,23.609,0.714,Congo (Democratic Republic Of The),Congo,Africa
121697,1856-03-01,24.878,0.742,Congo,Congo,Africa
119805,1856-04-01,23.601,0.743,Congo (Democratic Republic Of The),Congo,Africa
121698,1856-04-01,24.685,0.710,Congo,Congo,Africa
119806,1856-05-01,23.135,0.991,Congo (Democratic Republic Of The),Congo,Africa
121699,1856-05-01,24.038,0.981,Congo,Congo,Africa


In [ ]:
df.duplicated(subset=["CleanName", "dt"]).sum()

np.int64(13899)

There are 13,899 duplicate dates with temperatrues. we need to average them to make one

In [ ]:
df_grouped = (
    df[["CleanName", "dt", "Continent", "AverageTemperature", "AverageTemperatureUncertainty"]]
    .groupby(["CleanName", "dt", "Continent"], as_index=False)
    .agg({
        "AverageTemperature": "mean",                   # automatically skips NaN
        "AverageTemperatureUncertainty": "mean"         # automatically skips NaN
    })
)

In [ ]:
df_grouped["dt"] = pd.to_datetime(df_grouped["dt"])
df_grouped = df_grouped.sort_values(["CleanName", "dt"])

Reindex each country to FULL monthly frequency

Right now, if a month is missing, it simply does not exist in the table.

LSTMs require regular spacing, so we reconstruct the full monthly timeline before interpolation.

In [ ]:
import pandas as pd

full_groups = []

for country, g in df_grouped.groupby("CleanName"):
    g = g.set_index("dt").sort_index()

    # Create full monthly index from min to max
    full_index = pd.date_range(start=g.index.min(), end=g.index.max(), freq="MS")

    # Reindex (missing months become NaN)
    g = g.reindex(full_index)

    # Restore country and continent columns
    g["CleanName"] = country
    g["Continent"] = g["Continent"].ffill().bfill()  # safe — continent never changes

    full_groups.append(g)

df_full = pd.concat(full_groups).reset_index().rename(columns={"index": "dt"})

In [ ]:
df_full.head()

,dt,CleanName,Continent,AverageTemperature,AverageTemperatureUncertainty
0,1838-04-01,Afghanistan,Asia,13.008,2.586
1,1838-05-01,Afghanistan,Asia,NaN,NaN
2,1838-06-01,Afghanistan,Asia,23.950,2.510
3,1838-07-01,Afghanistan,Asia,26.877,2.883
4,1838-08-01,Afghanistan,Asia,24.938,2.992


Now we confirm we have no duplicate temperature values per dates

In [ ]:
df_full.duplicated(subset=["CleanName", "dt"]).sum()

np.int64(0)

We need to analyze missing data gap sizes

In [ ]:
df_full["is_nan"] = df_full["AverageTemperature"].isna()

df_full["gap_block"] = (df_full["is_nan"] != df_full["is_nan"].shift()).cumsum()
df_full["gap_size"] = df_full.groupby(["CleanName", "gap_block"])["is_nan"].transform("sum")

In [ ]:
df_full[df_full["is_nan"]].groupby("gap_size").size()

,0
gap_size,
1,1001
2,330
3,375
4,272
5,300
...,...
324,648
410,410
542,2168


Remove rows with large gaps

In [ ]:
df_no_big_gaps = df_full[df_full["gap_size"] <= 3].copy()

In [ ]:
df_no_big_gaps.shape

(533022, 8)

Small gap interpolation

In [ ]:
df_small = df_no_big_gaps.copy()

df_small["AverageTemperature"] = (
    df_small["AverageTemperature"]
    .interpolate(method="linear", limit_direction="both")
)

In [ ]:
df_small["AverageTemperature"].isna().sum()

np.int64(0)

In [ ]:
df_small.shape

(533022, 8)

In [ ]:
df_small["Continent"].value_counts()

,count
Continent,
Europe,157688
Asia,114516
Africa,105230
North America,90135
Oceania,32892
South America,29509
Antarctica,3052


In [ ]:
df_small

,dt,CleanName,Continent,AverageTemperature,AverageTemperatureUncertainty,is_nan,gap_block,gap_size
0,1838-04-01,Afghanistan,Asia,13.008,2.586,False,1,0
1,1838-05-01,Afghanistan,Asia,18.479,NaN,True,2,1
2,1838-06-01,Afghanistan,Asia,23.950,2.510,False,3,0
3,1838-07-01,Afghanistan,Asia,26.877,2.883,False,3,0
4,1838-08-01,Afghanistan,Asia,24.938,2.992,False,3,0
...,...,...,...,...,...,...,...,...
563558,2013-05-01,Åland,Europe,10.327,0.612,False,3697,0
563559,2013-06-01,Åland,Europe,14.068,0.423,False,3697,0
563560,2013-07-01,Åland,Europe,16.447,0.483,False,3697,0
563561,2013-08-01,Åland,Europe,16.425,0.378,False,3697,0


Save final dataset as csv

In [ ]:
df_small.to_csv("final_climate_data_with_interpolation.csv", index=False)